<a href="https://colab.research.google.com/github/anaribeiros/eurofootball/blob/main/EuroFootball.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Transfermarkt data scraper**

**Goal:** Scrape avg. market values, transfer expenses, previous season stats and attendance data from the most prominent clubs from Germany (Bundesliga), France (Ligue 1), Spain (La Liga), Italy (Serie A). All data was scraped from Transfermarkt, German-based website owned that has footballing information, such as scores, results, statistics, transfer news, and fixtures.

**Why?**
  * Context: a fellow Economics friend had started to manually scrape all this data to analyze for her Sports Economics class.
  * I really wanted her to get back to a healthy sleep schedule, so I decided to dedicate a bit of my time to this script so she could get her data (and her 8h of sleep a night)!

**What I've used**
*   Beautiful Soup: to scrape data out of Transfermarkt
*   Pandas: to create/manipulate a dataframe w/ football information
*   Xlsxwriter: to write results on an excel document



##**Setting up excel writer**

In [ ]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# creating an ExcelWriter object
writer = pd.ExcelWriter('sports_data.xlsx', engine='xlsxwriter')

##**Average market values**

**initial imports and data lists**

In [ ]:
# importing requests and beautiful soup
import requests
from bs4 import BeautifulSoup

In [ ]:
# generating list of seasons from 04/05 to 21/22
list_seasons = [str(i).zfill(2)+'/'+str(i+1).zfill(2) for i in range(4,22)]
list_seasons

['04/05',
 '05/06',
 '06/07',
 '07/08',
 '08/09',
 '09/10',
 '10/11',
 '11/12',
 '12/13',
 '13/14',
 '14/15',
 '15/16',
 '16/17',
 '17/18',
 '18/19',
 '19/20',
 '20/21',
 '21/22']

In [ ]:
# generating URLs based on base url
URLs = {'Germany':"https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=20",
        'France':"https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=20",
        'Italy':"https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=20",
        'Spain': "https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=20"}



In [ ]:
# user-agent to avoid 404 errors

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


**functions to run on each url**

In [ ]:
def return_urls(URL):
  return {year: URL+year[0:2] for year in list_seasons}

In [ ]:
# requesting url and converting it to a soup

def return_soup(url):
  page = requests.get(url, headers=headers)
  return BeautifulSoup(page.content, "html.parser")

In [ ]:
# getting teams' data table from transfer market

def table_teams(soup, season, country):
  # getting teams table content
  teams = soup.find(id='yw1').find(class_='items').find('tbody').find_all('tr')
  # data lists
  name = []
  age = []
  avg_mkt_val = []
  total_mkt_val = []
  season = len(teams) * [season]
  country = len(teams) * [country]
  for team in teams:
    name.append(team.find(class_='hauptlink no-border-links').find('a').text)
    age.append(team.find_all(class_='zentriert')[1].find('a').text)
    avg_mkt_val.append(team.find_all(class_='rechts')[0].text)
    total_mkt_val.append(team.find_all(class_='rechts')[1].find('a').text)

  return name, age, avg_mkt_val, total_mkt_val, season, country


In [ ]:
# declaring empty lists for our column values
names = []
ages = []
avg_mkt_vals = []
total_mkt_vals = []
seasons = []
countries = []

In [ ]:
# looping through all urls to get the data we need
for country in URLs:
  list_urls = return_urls(URLs[country])
  for year in list_urls:
    url = list_urls[year]
    print(url)
    soup = return_soup(url)
    season = year
    season_data = table_teams(soup,season, country)
    names += season_data[0]
    ages += season_data[1]

    avg_mkt_vals += season_data[2]
    total_mkt_vals += season_data[3]
    seasons += season_data[4]
    countries += season_data[5]
    print(avg_mkt_vals)


https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2004
['€6.04m', '€3.40m', '€2.72m', '€2.62m', '€2.56m', '€2.18m', '€2.39m', '€2.12m', '€2.12m', '€1.33m', '€1.23m', '€1.20m', '€1.05m', '€952k', '€925k', '€704k', '€701k', '€523k']
https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2005
['€6.04m', '€3.40m', '€2.72m', '€2.62m', '€2.56m', '€2.18m', '€2.39m', '€2.12m', '€2.12m', '€1.33m', '€1.23m', '€1.20m', '€1.05m', '€952k', '€925k', '€704k', '€701k', '€523k', '€6.35m', '€3.99m', '€3.05m', '€2.72m', '€2.46m', '€2.24m', '€2.35m', '€2.12m', '€1.92m', '€1.63m', '€1.47m', '€1.37m', '€1.18m', '€1.25m', '€1.22m', '€1.07m', '€995k', '€780k']
https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2006
['€6.04m', '€3.40m', '€2.72m', '€2.62m', '€2.56m', '€2.18m', '€2.39m', '€2.12m', '€2.12m', '€1.33m', '€1.23m', '€1.20m', '€1.05m', '€952k', '€925k', '€704k', '€701k', '€523k', '€6.35m', '€3.99m', '€3.05m', '€2

**converting data into dataframe**

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame(list(zip(names, ages, avg_mkt_vals, total_mkt_vals, seasons, countries)), columns=['Name', 'Age', 'Average Market Value', 'Total Market Value', 'Season', 'Country'])


In [ ]:
dataframe

,Name,Age,Average Market Value,Total Market Value,Season,Country
0,Bayern Munich,29,€6.04m,€175.30m,04/05,Germany
1,SV Werder Bremen,29,€3.40m,€98.48m,04/05,Germany
2,Bayer 04 Leverkusen,32,€2.72m,€86.93m,04/05,Germany
3,FC Schalke 04,33,€2.62m,€86.33m,04/05,Germany
4,VfB Stuttgart,31,€2.56m,€79.50m,04/05,Germany
...,...,...,...,...,...,...
1399,RCD Mallorca,41,€1.81m,€74.15m,21/22,Spain
1400,Elche CF,37,€1.97m,€72.98m,21/22,Spain
1401,Cádiz CF,37,€1.83m,€67.80m,21/22,Spain
1402,Rayo Vallecano,36,€1.87m,€67.20m,21/22,Spain


Notre Dame divides it's jobs into 3 categories: On Campus, Off Campus and Community Service. We'll create a dataframe to store the data on the job postings and to specify which type of job each one of them is.

**Creating an Excel spreadsheet w/ all the data**

In [ ]:
# writing the DataFrame to the Excel sheet
dataframe.to_excel(writer, sheet_name='Club Market Value', index=False)

##**Transfer expenses**

**initial imports and data lists**

In [ ]:
# importing requests and beautiful soup
import requests
from bs4 import BeautifulSoup

In [ ]:
# generating list of seasons from 04/05 to 21/22
list_seasons = [str(i).zfill(2)+'/'+str(i+1).zfill(2) for i in range(4,22)]
list_seasons

['04/05',
 '05/06',
 '06/07',
 '07/08',
 '08/09',
 '09/10',
 '10/11',
 '11/12',
 '12/13',
 '13/14',
 '14/15',
 '15/16',
 '16/17',
 '17/18',
 '18/19',
 '19/20',
 '20/21',
 '21/22']

In [ ]:
# generating URLs based on base url
URLs = {'Germany':"https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=YEAR&saison_id_bis=YEAR&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
        'France':"https://www.transfermarkt.com/ligue-1/einnahmenausgaben/wettbewerb/FR1/plus/0?ids=a&sa=&saison_id=YEAR&saison_id_bis=YEAR&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
        'Italy':"https://www.transfermarkt.com/serie-a/einnahmenausgaben/wettbewerb/IT1/plus/0?ids=a&sa=&saison_id=YEAR&saison_id_bis=YEAR&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0",
        'Spain': "https://www.transfermarkt.com/laliga/einnahmenausgaben/wettbewerb/ES1/plus/0?ids=a&sa=&saison_id=YEAR&saison_id_bis=YEAR&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0"}



In [ ]:
# user-agent to avoid 404 errors

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


**functions to run on each url**

In [ ]:
def return_urls(URL):
  return {year: URL.replace('YEAR','20'+year[0:2]) for year in list_seasons}

In [ ]:
# requesting url and converting it to a soup

def return_soup(url):
  page = requests.get(url, headers=headers)
  return BeautifulSoup(page.content, "html.parser")

In [ ]:
# getting teams' data table from transfer market

def table_teams(soup, season, country):
  # getting teams table content
  teams = soup.find(id='yw1').find(class_='items').find('tbody').find_all('tr')
  # data lists
  club = []
  expenditures = []
  arrivals = []
  income = []
  departures = []
  transfer_record = []
  season = len(teams) * [season]
  country = len(teams) * [country]
  for team in teams:
    club.append(team.find(class_='hauptlink no-border-links').find('a').text)
    expenditures.append(team.find(class_='rechts hauptlink redtext').text)
    arrivals.append(team.find_all(class_='zentriert')[2].text)
    income.append(team.find(class_='rechts hauptlink greentext').text)
    departures.append(team.find_all(class_='zentriert')[3].find('a').text)
    find_transfer = team.find_all(class_='rechts hauptlink')[0]
    if find_transfer.find('span'):
      find_transfer = find_transfer.find('span').text
    transfer_record.append(find_transfer)

  return club, expenditures, arrivals, income, departures, transfer_record, season, country


In [ ]:
# declaring empty lists for our column values
clubs = []
expenditures = []
arrivals = []
incomes = []
departures = []
transfer_records = []
seasons = []
countries = []

In [ ]:
# looping through all urls to get the data we need

for country in URLs:
  list_urls = return_urls(URLs[country])
  for year in list_urls:
      url = list_urls[year]
      print(url)
      soup = return_soup(url)
      season = year
      season_data = table_teams(soup,season, country)
      clubs +=  season_data[0]
      expenditures += season_data[1]
      arrivals += season_data[2]
      incomes += season_data[3]
      departures += season_data[4]
      transfer_records += season_data[5]
      seasons += season_data[6]
      countries += season_data[7]
      #print(avg_mkt_vals)


https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=2004&saison_id_bis=2004&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0
https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=2005&saison_id_bis=2005&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0
https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=2006&saison_id_bis=2006&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0
https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=2007&saison_id_bis=2007&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0
https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=2008&saison_id_bis=2008&nat=&pos=&altersklasse=&w_s=&leihe=&intern=0
https://www.transfermarkt.com/bundesliga/einnahmenausgaben/wettbewerb/L1/plus/0?ids=a&sa=&saison_id=2009&saison_id_bis=2009&nat=&pos=&altersklasse=&w_s=&le

**converting data into dataframe**

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame(list(zip(clubs, expenditures, arrivals, incomes, departures, transfer_records, seasons, countries)), columns=['Club', 'Expenditure', 'Arrivals', 'Income', 'Departures', 'Transfer Record', 'Season', 'Country'])


In [ ]:
dataframe

,Club,Expenditure,Arrivals,Income,Departures,Transfer Record,Season,Country
0,Bayern Munich,€25.75m,6,€1.03m,3,€-24.72m,04/05,Germany
1,Hamburger SV,€13.15m,14,€8.40m,17,€-4.75m,04/05,Germany
2,SV Werder Bremen,€9.00m,10,€6.45m,15,€-2.55m,04/05,Germany
3,VfL Wolfsburg,€8.50m,10,-,9,€-8.50m,04/05,Germany
4,FC Schalke 04,€4.65m,12,€50k,13,€-4.60m,04/05,Germany
...,...,...,...,...,...,...,...,...
1399,Real Betis Balompié,€3.25m,9,€14.00m,8,€10.75m,21/22,Spain
1400,Real Sociedad,€2.50m,7,-,9,€-2.50m,21/22,Spain
1401,Deportivo Alavés,€1.15m,24,€3.50m,22,€2.35m,21/22,Spain
1402,Levante UD,€875k,13,-,10,€-875k,21/22,Spain


**Creating an Excel spreadsheet w/ all the data**

In [ ]:
# writing the DataFrame to the Excel sheet
dataframe.to_excel(writer, sheet_name='Transfer Expenses', index=False)

##**Previous season stats**

**initial imports and data lists**

In [ ]:
# importing requests and beautiful soup
import requests
from bs4 import BeautifulSoup

In [ ]:
# generating list of seasons from 04/05 to 21/22
list_seasons = [str(i).zfill(2)+'/'+str(i+1).zfill(2) for i in range(3,21)]
list_seasons

['03/04',
 '04/05',
 '05/06',
 '06/07',
 '07/08',
 '08/09',
 '09/10',
 '10/11',
 '11/12',
 '12/13',
 '13/14',
 '14/15',
 '15/16',
 '16/17',
 '17/18',
 '18/19',
 '19/20',
 '20/21']

In [ ]:
# generating URLs based on base url
URLs = {'Germany':"https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=20",
        'France':"https://www.transfermarkt.com/ligue-1/tabelle/wettbewerb/FR1?saison_id=20",
        'Italy':"https://www.transfermarkt.com/serie-a/tabelle/wettbewerb/IT1?saison_id=20",
        'Spain': "https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1?saison_id=20"}



In [ ]:
# user-agent to avoid 404 errors

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


**functions to run on each url**

In [ ]:
def return_urls(URL):
  return {year: URL+year[0:2] for year in list_seasons}

In [ ]:
# requesting url and converting it to a soup

def return_soup(url):
  page = requests.get(url, headers=headers)
  return BeautifulSoup(page.content, "html.parser")

In [ ]:
# getting teams' data table from transfer market

def table_teams(soup, season, country):
  # getting teams table content
  teams = soup.find(id='yw1').find(class_='items').find('tbody').find_all('tr')
  # data lists
  position = []
  club = []
  n_matches = []
  wins = []
  ties = []
  losses = []
  plus_minus = []
  points = []
  season = len(teams) * [season]
  country = len(teams) * [country]
  for team in teams:
    position.append(team.find(class_='rechts hauptlink').text)
    club.append(team.find(class_='no-border-links hauptlink').find('a').text)
    n_matches.append(team.find_all(class_='zentriert')[1].text)
    wins.append(team.find_all(class_='zentriert')[2].text)
    ties.append(team.find_all(class_='zentriert')[3].text)
    losses.append(team.find_all(class_='zentriert')[4].text)
    plus_minus.append(team.find_all(class_='zentriert')[6].text)
    points.append(team.find_all(class_='zentriert')[7].text)


  return position, club, n_matches, wins, ties, losses, plus_minus, points, season, country


In [ ]:
# declaring empty lists for our column values
positions = []
clubs = []
n_matches = []
wins = []
ties = []
losses = []
plus_minus = []
points = []
seasons = []
countries = []

In [ ]:
# looping through all urls to get the data we need

#for country in URLs:
for country in URLs:
  list_urls = return_urls(URLs[country])
  for year in list_urls:
        url = list_urls[year]
        print(url)
        soup = return_soup(url)
        season = year
        season_data = table_teams(soup,season, country)
        positions +=  season_data[0]
        clubs += season_data[1]
        n_matches += season_data[2]
        wins += season_data[3]
        ties += season_data[4]
        losses += season_data[5]
        plus_minus += season_data[6]
        points += season_data[7]
        seasons += season_data[8]
        countries += season_data[9]
        #print(avg_mkt_vals)


https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2003
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2004
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2005
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2006
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2007
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2008
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2009
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2010
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2011
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2012
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2013
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?saison_id=2014
https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1?s

**converting data into dataframe**

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame(list(zip(positions, clubs, n_matches, wins, ties, losses, plus_minus, points, seasons, countries)), columns=['Position', 'Club', 'Matches', 'Wins', 'Ties', 'Losses', '+/-', 'Points','Season', 'Country'])


In [ ]:
dataframe

,Position,Club,Matches,Wins,Ties,Losses,+/-,Points,Season,Country
0,1,Werder Bremen,34,22,8,4,41,74,03/04,Germany
1,2,Bayern Munich,34,20,8,6,31,68,03/04,Germany
2,3,B. Leverkusen,34,19,8,7,34,65,03/04,Germany
3,4,VfB Stuttgart,34,18,10,6,28,64,03/04,Germany
4,5,VfL Bochum,34,15,11,8,18,56,03/04,Germany
...,...,...,...,...,...,...,...,...,...,...
1397,16,Alavés,38,9,11,18,-21,38,20/21,Spain
1398,17,Elche CF,38,8,12,18,-21,36,20/21,Spain
1399,18,SD Huesca,38,7,13,18,-19,34,20/21,Spain
1400,19,Real Valladolid,38,5,16,17,-23,31,20/21,Spain


Notre Dame divides it's jobs into 3 categories: On Campus, Off Campus and Community Service. We'll create a dataframe to store the data on the job postings and to specify which type of job each one of them is.

**Creating an Excel spreadsheet w/ all the data**

In [ ]:
# writing the DataFrame to the Excel sheet
dataframe.to_excel(writer, sheet_name='Standings in Previous Seasons', index=False)

##**Attendance data**

**initial imports and data lists**

In [ ]:
# importing requests and beautiful soup
import requests
from bs4 import BeautifulSoup

In [ ]:
# generating list of seasons from 04/05 to 21/22
list_seasons = [str(i).zfill(2)+'/'+str(i+1).zfill(2) for i in range(4,22)]
list_seasons

['04/05',
 '05/06',
 '06/07',
 '07/08',
 '08/09',
 '09/10',
 '10/11',
 '11/12',
 '12/13',
 '13/14',
 '14/15',
 '15/16',
 '16/17',
 '17/18',
 '18/19',
 '19/20',
 '20/21',
 '21/22']

In [ ]:
# generating URLs based on base url
URLs = {'Germany':"https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=20",
        'France':"https://www.transfermarkt.com/ligue-1/besucherzahlen/wettbewerb/FR1/plus/1?saison_id=20",
        'Italy':"https://www.transfermarkt.com/serie-a/besucherzahlen/wettbewerb/IT1/plus/1?saison_id=20",
        'Spain': "https://www.transfermarkt.com/laliga/besucherzahlen/wettbewerb/ES1/plus/1?saison_id=20",
        'Great Britain': "https://www.transfermarkt.com/premier-league/besucherzahlen/wettbewerb/GB1/plus/1?saison_id=20"}



In [ ]:
# user-agent to avoid 404 errors

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


**functions to run on each url**

In [ ]:
def return_urls(URL):
  return {year: URL+year[0:2] for year in list_seasons}

In [ ]:
# requesting url and converting it to a soup

def return_soup(url):
  page = requests.get(url, headers=headers)
  return BeautifulSoup(page.content, "html.parser")

In [ ]:
# getting teams' data table from transfer market

def table_teams(soup, season, country):
  # getting teams table content
  teams = soup.find(id='yw1').find(class_='items').find('tbody').findChildren('tr', recursive=False)
  #print(teams)
  # data lists
  ranking = []
  stadium = []
  club = []
  capacity = []
  spectators = []
  average = []
  matches = []
  soldout = []
  cap_percent = []
  season = len(teams) * [season]
  country = len(teams) * [country]
  for team in teams:

    ranking.append(team.find(class_='zentriert').text)
    data_stadium = team.find(class_='inline-table')
    stadium.append(data_stadium.find('tr').find_all('td')[1].find('a').text)
    club.append(data_stadium.find_all('tr')[1].find('td').find('a').text)
    capacity.append(team.find(class_='rechts').text)
    spectators.append(team.find_all(class_='rechts')[1].text)
    average.append(team.find_all(class_='rechts')[2].text)
    matches.append(team.find_all(class_='zentriert')[2].text)
    soldout.append(team.find_all(class_='zentriert')[3].text)
    cap_percent.append(team.find(class_='rechts hauptlink').text)


  return ranking, stadium, club, capacity, spectators, average, matches, soldout, cap_percent, season, country



In [ ]:
# declaring empty lists for our column values
rankings = []
stadiums = []
clubs = []
capacity = []
spectators = []
average = []
matches = []
soldout = []
cap_percent = []
seasons = []
countries = []

In [ ]:
# looping through all urls to get the data we need

for country in URLs:
  list_urls = return_urls(URLs[country])
  for year in list_urls:
        url = list_urls[year]
        print(url)
        soup = return_soup(url)
        season = year
        season_data = table_teams(soup,season, country)
        rankings +=  season_data[0]
        stadiums += season_data[1]
        clubs += season_data[2]
        capacity += season_data[3]
        spectators += season_data[4]
        average += season_data[5]
        matches += season_data[6]
        soldout += season_data[7]
        cap_percent += season_data[8]
        seasons += season_data[9]
        countries += season_data[10]
        #print(avg_mkt_vals)


https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2004
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2005
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2006
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2007
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2008
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2009
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2010
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2011
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2012
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?saison_id=2013
https://www.transfermarkt.com/bundesliga/besucherzahlen/wettbewerb/L1/plus/1?sai

**converting data into dataframe**

In [ ]:
import pandas as pd

In [ ]:
dataframe = pd.DataFrame(list(zip(rankings, stadiums, clubs, capacity, spectators, average, matches, soldout, cap_percent, seasons, countries)), columns=['Position', 'Stadium', 'Club', 'Capacity', 'Spectators', 'Average', 'Matches', 'Soldout', 'Capacity (%)', 'Season', 'Country'])


In [ ]:
dataframe

,Position,Stadium,Club,Capacity,Spectators,Average,Matches,Soldout,Capacity (%),Season,Country
0,1,SIGNAL IDUNA PARK,Borussia Dortmund,81.365,1.313.000,77.235,17,2,94.9 %,04/05,Germany
1,2,Veltins-Arena,FC Schalke 04,62.271,1.043.962,61.410,17,14,98.6 %,04/05,Germany
2,3,Allianz Arena,Bayern Munich,75.024,906.000,53.294,17,2,71.0 %,04/05,Germany
3,4,Stadion im Borussia-Park,Borussia Mönchengladbach,54.042,835.863,49.168,17,7,91.0 %,04/05,Germany
4,5,Volksparkstadion,Hamburger SV,57.000,831.769,48.928,17,4,85.8 %,04/05,Germany
...,...,...,...,...,...,...,...,...,...,...,...
1759,16,St Mary's Stadium,Southampton FC,32.384,448.223,29.882,15,-,92.3 %,21/22,Great Britain
1760,17,Carrow Road,Norwich City,27.244,430.124,26.883,16,-,98.7 %,21/22,Great Britain
1761,18,Vicarage Road,Watford FC,21.577,391.369,20.598,19,-,95.5 %,21/22,Great Britain
1762,19,Turf Moor,Burnley FC,21.994,347.708,19.317,18,-,87.8 %,21/22,Great Britain


Notre Dame divides it's jobs into 3 categories: On Campus, Off Campus and Community Service. We'll create a dataframe to store the data on the job postings and to specify which type of job each one of them is.

**Creating an Excel spreadsheet w/ all the data**

In [ ]:
# writing the DataFrame to the Excel sheet
dataframe.to_excel(writer, sheet_name='Attendance Data', index=False)

## **Saving excel file**

In [ ]:
# save the Excel file
writer.save()